In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import skimage.io as io

from PIL import Image

import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2

from pathlib import Path

import sklearn.feature_extraction
import sklearn.feature_extraction
from skimage.segmentation import expand_labels, watershed
import skimage.morphology

from sklearn.cluster import spectral_clustering

from skimage import morphology


import deepdrr
from deepdrr import Volume
from deepdrr import geo
from deepdrr.utils import test_utils

import nibabel as nib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/pulmonary_vein.nii.gz")
vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.nii.gz")
mesh_path = Path("/home/pelvisvr/PrephixData/ct_meshes/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.stl")

/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.nii.gz
True
/home/pelvisvr/PrephixData/ct_meshes/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.stl
True


In [5]:
vol = Volume.from_nifti(vol_path, segmentation=True)
vol.shape
vol_poly = vol.isosurface(decimation=0, smooth_iter=0, smooth=False,)
vol_poly.save("mesh.stl")
print(vol_poly.bounds)
vol_cropped = vol.shrink(padding=5)
vol_cropped.shape
vol_poly = vol_cropped.isosurface(decimation=0, smooth_iter=0, smooth=False,)
vol_poly.save("cropped_mesh.stl")
print(vol_poly.bounds)

bodies = vol_cropped.split_bodies()
vol_data = vol_cropped.data

# vol_data = morphology.binary_erosion(vol_data, footprint=morphology.ball(5))
vol_data = morphology.binary_dilation(vol_data, footprint=morphology.ball(5))
float_labels = vol_data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas2.nii.gz')
body = bodies[0]

body.shape
body_shrunk = body.shrink(padding=5)
print(body_shrunk.shape)
shrunk_downscaled = body_shrunk.downscale((3,3,1), segmentation=True)
print(shrunk_downscaled.shape)

float_labels = shrunk_downscaled.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas3.nii.gz')
segmented = shrunk_downscaled.cluster_seg(segmentation=True, n_clusters=9, expand=10)

float_labels = segmented.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas4.nii.gz')
segmented_upscaled = segmented.upscale((3,3,1), segmentation=True, smooth_seg=(3,3,3))

float_labels = segmented_upscaled.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas5.nii.gz')
print(segmented_upscaled.shape)
print(body_shrunk.shape)
split_parts = body_shrunk.split_by_segmentation(segmented_upscaled.data)
print(len(split_parts))

In [81]:
# zip parts
import zipfile
with zipfile.ZipFile("parts.zip", "w") as zf:
    for i, part in enumerate(split_parts):
        part_poly = part.isosurface(decimation_points=20000,smooth=False,convert_to_LPS=True,)
        # part_poly = part.isosurface(decimation=0, smooth_iter=0, smooth=False,)
        part_poly.save(f"part_{i}.stl")
        zf.write(f"part_{i}.stl")